In [ ]:
import colorsys as cs
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image,ImageEnhance
from skimage import data
from scipy import misc
 

In [ ]:
source = Image.open('cat_test.jpeg')
template = Image.open('cat_test2.jpg') 
sd = data.coffee()
np.asarray(source)

In [ ]:
rgb_image_source = np.asarray(source)
hls_image = [[ list(cs.rgb_to_hls(*rgb/255)) for rgb in row] for row in rgb_image]

In [ ]:
def hls_to_rgb(r, g, b):
  return[ color * 255 for color in list(cs.hls_to_rgb(r,g,b))]
 

def adjust_hls(image, hue=1, light=1, saturation=1):
  hls_image = [[list(cs.rgb_to_hls(*rgb/255)) for rgb in row] for row in image]
  hls_adjusted_image = [[[hls[0]*hue,hls[1]*light,hls[2]*saturation] for hls in row] for row in hls_image]
  back_to_rgb = [[ np.asarray(hls_to_rgb(*rgb)).astype(int) for rgb in row] for row in hls_adjusted_image]
  return back_to_rgb

def match_cdf(source):
  values,unique_indices,counts = np.unique(source.ravel(),return_inverse = True, return_counts = True)
  return values,unique_indices,counts
  

In [ ]:
def _match_cumulative_cdf(source, template):
    """
    Return modified source array so that the cumulative density function of
    its values matches the cumulative density function of the template.
    """
    rgb_image_source = np.array(source)
    rgb_image_template = np.array(template)
    src_values, src_unique_indices, src_counts = np.unique(rgb_image_source.ravel(),return_inverse=True,return_counts=True)
    tmpl_values, tmpl_counts = np.unique(rgb_image_template.ravel(), return_counts=True)

    # calculate normalized quantiles for each array
    src_quantiles = np.cumsum(src_counts) / rgb_image_source.size
    tmpl_quantiles = np.cumsum(tmpl_counts) / rgb_image_template.size

    # print(src_quantiles)

    interp_a_values = np.interp(tmpl_values, src_quantiles,tmpl_quantiles)
    print(interp_a_values)
    print(interp_a_values[src_unique_indices])
    return interp_a_values[src_unique_indices].reshape(rgb_image_source.shape)

In [ ]:
source1 = Image.open('source.png')
reference = Image.open('reference.png')
source = Image.open('cat_test.jpeg')
template = Image.open('cat_test2.jpg') 
matched = _match_cumulative_cdf(source, source)
# out_dtype = utils._supported_float_type(image.dtype)

#matched_rgb = [[ list(cs.hls_to_rgb(*hls)) for hls in row] for row in matched]
#matched.astype(out_dtype, copy=False)
plt.imshow(matched)


In [ ]:
source1 = misc.imread('reference.png')

In [ ]:
rgb_bright = adjust_hls(rgb_image)
image2 = plt.imshow(rgb_bright)
image2

In [ ]:
""" The scikit histogram_matching function reads in a numpy array of RGB values and creates an average of the values
Then uses the np.interp and .reshape to normalize the values. Instead of reading in RGB values, we could use the 
colorsys library to convert the RGB values to HSL values (hue, saturation, lightness). We can then pull each individual
value to create separate tools for Hue, Saturation, and Lightness."""

In [ ]:
def calculate_cdf(histogram):
    """
    This method calculates the cumulative distribution function
    :param array histogram: The values of the histogram
    :return: normalized_cdf: The normalized cumulative distribution function
    :rtype: array
    """
    # Get the cumulative sum of the elements
    cdf = histogram.cumsum()
 
    # Normalize the cdf
    normalized_cdf = cdf / float(cdf.max())
 
    return normalized_cdf

def calculate_lookup(src_cdf, ref_cdf):
    """
    This method creates the lookup table
    :param array src_cdf: The cdf for the source image
    :param array ref_cdf: The cdf for the reference image
    :return: lookup_table: The lookup table
    :rtype: array
    """
    lookup_table = np.zeros(256)
    lookup_val = 0
    for src_pixel_val in range(len(src_cdf)):
        lookup_val
        for ref_pixel_val in range(len(ref_cdf)):
            if ref_cdf[ref_pixel_val] >= src_cdf[src_pixel_val]:
                lookup_val = ref_pixel_val
                break
        lookup_table[src_pixel_val] = lookup_val
    return lookup_table
    
def match_histograms(src_image, ref_image):
    """
    This method matches the source image histogram to the
    reference signal
    :param image src_image: The original source image
    :param image  ref_image: The reference image
    :return: image_after_matching
    :rtype: image (array)
    """
    # Split the images into the different color channels
    # b means blue, g means green and r means red
    src_b, src_g, src_r = cv2.split(src_image)
    ref_b, ref_g, ref_r = cv2.split(ref_image)
 
    # Compute the b, g, and r histograms separately
    # The flatten() Numpy method returns a copy of the array c
    # collapsed into one dimension.
    src_hist_blue, bin_0 = np.histogram(src_b.flatten(), 256, [0,256])
    src_hist_green, bin_1 = np.histogram(src_g.flatten(), 256, [0,256])
    src_hist_red, bin_2 = np.histogram(src_r.flatten(), 256, [0,256])    
    ref_hist_blue, bin_3 = np.histogram(ref_b.flatten(), 256, [0,256])    
    ref_hist_green, bin_4 = np.histogram(ref_g.flatten(), 256, [0,256])
    ref_hist_red, bin_5 = np.histogram(ref_r.flatten(), 256, [0,256])
 
    # Compute the normalized cdf for the source and reference image
    src_cdf_blue = calculate_cdf(src_hist_blue)
    src_cdf_green = calculate_cdf(src_hist_green)
    src_cdf_red = calculate_cdf(src_hist_red)
    ref_cdf_blue = calculate_cdf(ref_hist_blue)
    ref_cdf_green = calculate_cdf(ref_hist_green)
    ref_cdf_red = calculate_cdf(ref_hist_red)
 
    # Make a separate lookup table for each color
    blue_lookup_table = calculate_lookup(src_cdf_blue, ref_cdf_blue)
    green_lookup_table = calculate_lookup(src_cdf_green, ref_cdf_green)
    red_lookup_table = calculate_lookup(src_cdf_red, ref_cdf_red)
 
    # Use the lookup function to transform the colors of the original
    # source image
    blue_after_transform = cv2.LUT(src_b, blue_lookup_table)
    green_after_transform = cv2.LUT(src_g, green_lookup_table)
    red_after_transform = cv2.LUT(src_r, red_lookup_table)
 
    # Put the image back together
    image_after_matching = cv2.merge([
        blue_after_transform, green_after_transform, red_after_transform])
    image_after_matching = cv2.convertScaleAbs(image_after_matching)
 
    return image_after_matching